In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import xgboost as xgb

In [3]:
test = pd.read_csv("data/test_with_predicted_lat_lon.csv.gz")
test.head()

,merchant_id,latitude,longitude,real_transaction_dttm,record_delay,dist,is_in_area,longitude_origin,latitude_origin
0,1150899,46.498337,13.507997,39293.0,0.0,2517926.0,0,40.202568,47.123615
1,411612,56.117978,47.475212,42592.0,0.0,45041.0,1,47.877750,55.748322
2,616963,31.781375,2.905627,67258.0,0.0,4353273.0,0,35.446640,54.036610
3,419934,60.000674,30.264237,75541.0,0.0,17709.0,1,30.491623,59.912495
4,407995,55.708998,37.620635,73061.0,0.0,6397.0,1,37.552544,55.744450


In [4]:
from geopy.distance import vincenty

def distance(data):
    for i, row in data.iterrows():
        data.ix[i, 'dist_predicted'] = vincenty((row['latitude'], row['longitude']), (row['latitude_origin'], row['longitude_origin'])).meters

In [5]:
distance(test)

,merchant_id,latitude,longitude,real_transaction_dttm,record_delay,dist,is_in_area,longitude_origin,latitude_origin,dist_predicted
0,1150899,46.498337,13.507997,39293.0,0.0,2517926.0,0,40.202568,47.123615,2.028696e+06
1,411612,56.117978,47.475212,42592.0,0.0,45041.0,1,47.877750,55.748322,4.823811e+04
2,616963,31.781375,2.905627,67258.0,0.0,4353273.0,0,35.446640,54.036610,3.572543e+06
3,419934,60.000674,30.264237,75541.0,0.0,17709.0,1,30.491623,59.912495,1.606006e+04
4,407995,55.708998,37.620635,73061.0,0.0,6397.0,1,37.552544,55.744450,5.820972e+03


In [7]:
test['dist_predicted'] = np.round(test['dist_predicted'])
test.head()

,merchant_id,latitude,longitude,real_transaction_dttm,record_delay,dist,is_in_area,longitude_origin,latitude_origin,dist_predicted
0,1150899,46.498337,13.507997,39293.0,0.0,2517926.0,0,40.202568,47.123615,2028696.0
1,411612,56.117978,47.475212,42592.0,0.0,45041.0,1,47.877750,55.748322,48238.0
2,616963,31.781375,2.905627,67258.0,0.0,4353273.0,0,35.446640,54.036610,3572543.0
3,419934,60.000674,30.264237,75541.0,0.0,17709.0,1,30.491623,59.912495,16060.0
4,407995,55.708998,37.620635,73061.0,0.0,6397.0,1,37.552544,55.744450,5821.0


In [17]:
# может будет проще выделять нужное потом.
# Теоретически сузить область значений. Тогда будет больше повторов и можно будет сортировать каскадно по другим индексам.
test['dist_norm'] = test['dist_predicted'] // test['dist_predicted'].min()

In [20]:
test.sort_values(['merchant_id', 'dist_norm', 'record_delay'], inplace=True)
test.head()

,merchant_id,latitude,longitude,real_transaction_dttm,record_delay,dist,is_in_area,longitude_origin,latitude_origin,dist_predicted,dist_norm
16581,361,55.750347,37.623851,43179.0,112.0,930427.0,1,34.455822,55.874847,199083.0,6032.0
34192,361,55.750347,37.623511,41004.0,352.0,903222.0,1,34.455822,55.874516,199060.0,6032.0
21265,361,55.444675,36.465961,27646.0,158.0,521262.0,1,29.614254,59.567509,615437.0,18649.0
36457,361,56.824988,60.675723,58304.0,401.0,564168.0,1,50.491112,55.202450,659702.0,19990.0
38677,361,56.861649,60.665363,63788.0,453.0,519475.0,1,48.268215,55.210983,793114.0,24033.0


In [27]:
result = pd.DataFrame(test[test.duplicated(['merchant_id']) == False])
print(result.shape)
result.head()

(3524, 11)


,merchant_id,latitude,longitude,real_transaction_dttm,record_delay,dist,is_in_area,longitude_origin,latitude_origin,dist_predicted,dist_norm
16581,361,55.750347,37.623851,43179.0,112.0,930427.0,1,34.455822,55.874847,199083.0,6032.0
16394,428,55.553254,37.723753,64036.0,111.0,457550.0,1,31.195684,58.889050,541339.0,16404.0
41045,490,59.875699,30.361785,61535.0,514.0,147969.0,1,31.056566,60.838299,113900.0,3451.0
32198,1175,55.733561,37.664147,73917.0,314.0,80227.0,1,37.829803,55.749718,10559.0,319.0
5760,1239,55.750347,37.623851,51381.0,35.0,568301.0,1,37.950016,55.200272,64621.0,1958.0


In [30]:
merch_test = pd.read_csv('data/merchants_test.csv.zip', delimiter=';')
print(merch_test.shape)
merch_test.head()

(3524, 3)


,merchant_id,latitude,longitude
0,361,NaN,NaN
1,428,NaN,NaN
2,490,NaN,NaN
3,1175,NaN,NaN
4,1239,NaN,NaN


In [43]:
def save(result, fname):
    submission = pd.DataFrame()
    submission['merchant_id'] = result['merchant_id']
    submission['latitude'] = result['latitude_origin']
    submission['longitude'] = result['longitude_origin']
    submission.to_csv('submissions/{}.csv'.format(fname), index=False)

In [45]:
save(result, 'sub_1')

In [48]:
res2 = test.groupby('merchant_id', as_index=False)[['longitude_origin', 'latitude_origin']].median()
res2.head()

,merchant_id,longitude_origin,latitude_origin
0,361,39.709412,55.615692
1,428,31.170138,59.311680
2,490,34.286732,58.322540
3,1175,37.244858,55.707325
4,1239,35.937458,55.855440


In [49]:
save(res2, 'sub_2')

На самом деле это полный фейл. Брать медианы - это вообще 0. А брать самое близкое - 0.0009 - вообще смешно, аж стыдно.